<a href="https://colab.research.google.com/github/nithinivi/Deep_Learning_Discussion/blob/main/002_Convnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolution Neural Networks


- kamming paper (Imagenet )

- what convolutions layer are learning (Fergerson Paper)

- Max Pooling 

- batch Normalization 




![Convolution](https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/padding_strides.gif "Conv With Stride and padding")